In [ ]:
import pandas as pd
import evolven

# Report: Identify common modifications to new VMs

The purpose of the report is to identify additional areas to be configured out-of-the-box in a new server build
 - Users can order a new VM that will come with one of standard builds
 - Once VM is ready the users can add missing software and execute additional configuration
 - If most of the users are doing the same changes our sponsor wants to detect this and add these changes to a standard build
    
Scope of the report can include
 - An environment, a list of environments or a list of rules defining a set of environments
 - A list of servers or a list of rules defining sets of servers

Period of time will be a number of days since the first agent scan.

Outcome will provide statistics of the changes in the defined scope over defined period of time:
 - Top 5 technology components added the most on top of the standard builds + a list of added components and percentage of servers where these components were deployed
 - Top 10 configuration changes made across the target scope + an ordered list of changes executed on more than 5% of servers (the percentage should be configurable)
 - A list of version updates per component executed on the servers after the initial build

## Pull all relevant data

In three steps:
1. Initialize Evolven API object
2. Retrieve all relevant hosts
4. Retrieve all changes on that hosts

### Step 1: Initialize Evolven API object

In [ ]:
api = evolven.EvolvenAPI("http://localhost", port="8080", 
                            username="...", password="...", 
                            return_type="DataTable", debug=False)

### Step 2: Retrieve all discovered hosts

... and select a subset of hosts

In [ ]:
# Retrieve all discovered hosts
hosts = api.Host.list()

# Select a subset of hosts, e.g., production hosts
selected_hosts = hosts[hosts.HostType=="Prod"]
# selected_hosts = hosts[hosts.Name.str.contains("US3_GSP_PROD")]

print("Total selected hosts", len(selected_hosts))

# Test print the first 2 hosts
selected_hosts.head(2)
# selected_hosts

### Step 3: Retrieve all changes at selected hosts

In [ ]:
changes = pd.DataFrame()

for idx, host in selected_hosts[["Name", "envId", "ID"]].iterrows():
    print(host.Name)
    changes_h = api.Change.list(env_id=host["envId"])
    print(len(changes_h))
    changes = changes.append(changes_h)
    
    
print("\nTotal changes %s"%len(changes))

# Print all change fields
print("\nColumns:\n" + ", ".join([col for col in changes]))

# Test print the first 2 changes
changes.head(2)
# changes

### Step 4: Get age of changes
Calculate how much time passed between the time the machine was created and change was introduced.

In [ ]:
age = []
host_t_cache = {}

for idx, change in changes.iterrows():
    
    c_age = 0
    
    # Get the right host name
    host = hosts[hosts.Name==change.Host]
    if len(host) > 0:
        env_id = int(host.envId)

        # Query environment info for this host
        if env_id in host_t_cache:
            host_t_created = host_t_cache[env_id]
        else:        
            host_t_created = int(api.Environment.get_summary(env_id).Next.CreationTime)
            host_t_cache[env_id] = host_t_created

        # Calculate relative time difference
        change_t_created = int(change.StartTime)
        c_age = (change_t_created - host_t_created)
    else:
        # error in demo data
        pass
        
    age.append(c_age)

    

# Store data and calculate age in days and weeks
changes["age"] = age
changes["days"] = [a/1000/3600/24 for a in age]
changes["weeks"] = [d/7 for d in changes["days"]]
    
changes.head(2)

## Address report objectives
Outcome will provide statistics of the changes in the defined scope over defined period of time
1. Top 5 technology components added the most on top of the standard builds + a list of added components and percentage of servers where these components were deployed
1. Top 10 configuration changes made across the target scope + an ordered list of changes executed on more than 5% of servers (the percentage should be configurable)
1. A list of version updates per component executed on the servers after the initial build

### 1. Top 5 technology components added the most on top of the standard builds + a list of added components and percentage of servers where these components were deployed

First select only inventory changes

In [ ]:
inventory_changes = changes[changes["CIType"]=="Inventory"]
print("Total", len(inventory_changes))

In [ ]:
# Get total servers
total_servers = inventory_changes["hostId"].nunique()
print("Total servers", total_servers)

Group changes by Environment type and full path, and count the number of changes. Then add a column that calculates the share of servers.

In [ ]:
# SELECT COUNT(ID) AS ID, EnvironmentType, FullPath 
# FROM inventory_changes 
# GROUP BY EnvironmentType, FullPath 
# SORT BY ID

inventory_grouped = inventory_changes.groupby(["EnvironmentType", "FullPath", "Operation"])["ID"].count().reset_index()
inventory_grouped = inventory_grouped.sort_values("ID", ascending=False)

inventory_grouped["Share of servers"] = inventory_grouped["ID"] * 100.0 / total_servers

# Test print the first 5 changes
inventory_grouped.head()

### 2. Top 10 configuration changes made across the target scope + an ordered list of changes executed on more than 5% of servers (the percentage should be configurable)

Start by selecting configuration and capacity changes.

In [ ]:
configuration_changes = changes[changes["CIType"].isin(["Configuration", "Capacity"])]
print("Total changes", len(configuration_changes))
configuration_changes.head(2)

total_servers = configuration_changes["hostId"].nunique()
print("Total servers", total_servers)

In [ ]:
# SELECT COUNT(ID) AS ID, EnvironmentType, FullPath, AnnotatedDescription
# FROM configuration_changes 
# GROUP BY EnvironmentType, FullPath, AnnotatedDescription 
# SORT BY ID

config_grouped = configuration_changes.groupby(["EnvironmentType", "FullPath", "AnnotatedDescription", "CISubType"])["ID"].count().reset_index()
config_grouped = config_grouped.sort_values("ID", ascending=False)

config_grouped["Share of servers"] = config_grouped["ID"] * 100.0 / total_servers

# Show only configuration changes above threshold
config_grouped = config_grouped[config_grouped["Share of servers"] > 5.0]

# Test print the first 5 changes
config_grouped.head(5)

### 3. A list of version updates per component executed on the servers after the initial build

In [ ]:
version_changes = changes[changes["value-type"] == "VERSION"]
print("Total changes", len(version_changes))
version_changes.head(2)

total_servers = version_changes["hostId"].nunique()
print("Total servers", total_servers)

# SELECT COUNT(ID) AS ID, EnvironmentType, FullPath, AnnotatedDescription 
# FROM version_changes 
# GROUP BY EnvironmentType, FullPath, AnnotatedDescription 
# SORT BY ID

version_grouped = version_changes.groupby(["EnvironmentType", "FullPath", "AnnotatedDescription"])["ID"].count().reset_index()
version_grouped = version_grouped.sort_values("ID", ascending=False)

version_grouped["Share of servers"] = version_grouped["ID"] * 100.0 / total_servers

# Show only configuration changes above threshold
version_grouped = version_grouped[version_grouped["Share of servers"] > 5.0]

# Test print the first 5 changes
version_grouped.head(5)

## Write results to an Excel file
Create a new Excel file and write each result on a its own sheet.

In [ ]:
path = "C:\\Users\\Administrator\\Downloads\\evolven-python-quickstart\\scripts\\03-bk-inventory-analysis"
report_path = path + '\\inventory-analysis.xlsx'

writer = pd.ExcelWriter(report_path)

inventory_grouped.to_excel(writer,'Top inventory changes')
config_grouped.to_excel(writer,'Top configuration changes')
version_grouped.to_excel(writer,'Top version changes')

writer.save()

## Set output variables for Evolven reporting
Detailed set of input and output variables are described in [00 - Advanced Reporting with Evolven Python API.ipynb](00 - Advanced Reporting with Evolven Python API.ipynb).

In [ ]:
## Set variables for
import os, sys


print("ReportLocation=%s"%report_path)
print("Subject=Inventory analysis")
print("Body=Inventory analysis attached. Evolven Python Report")
